In [94]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sbn
from tqdm import tqdm

sbn.set()

<h3>(1) Crude Monte Carlo estimate </h3>

In [78]:
n = 10000

actual_integral = np.exp(1) - 1

U = np.random.uniform(0, 1, size = n)

In [34]:
n = 1000

X = np.exp(U)

X_bar = np.mean(X)

error = np.abs(X_bar - actual_integral) * 100

print(f'Integral estimate Monte Carlo: {X_bar}, actual: {actual_integral}, error = {error:.2f}%')

Integral estimate Monte Carlo: 1.7178253761791937, actual: 1.718281828459045, error = 0.05%


<h3>(2) Antithetic estimate </h3>

In [44]:
X = np.exp(U)

Y = (np.exp(U) + np.exp(1 - U)) / 2

Y_bar = np.mean(Y)

error = np.abs(Y_bar - actual_integral)

print(f'Integral estimate Antithetic: {Y_bar}, actual: {actual_integral}, error = {error * 100:.2f}%')
print(f'Variance reduction = {((np.abs(np.std(Y) ** 2 - np.std(X) ** 2)) / np.std(X) ** 2) * 100:.2f}%')

Integral estimate Antithetic: 1.7186718932354077, actual: 1.718281828459045, error = 0.04%
Variance reduction = 98.38%


The estimate of the integral using antithetic variables is a little bit more precise and reduced the variance with >98 %.

<h3>(3) Control variate estimate</h3>

In [51]:
X = np.exp(U)

c = np.mean(U * X) - np.mean(U) * np.mean(X)

Z = X + c*(U - 1/2)
Z_bar = np.mean(Z)

error = np.abs(Z_bar - actual_integral)

print(f'Integral estimate Control Variate: {Z_bar:.5f}, actual: {actual_integral:.5f}, error = {error * 100:.2f}%')
print(f'Variance reduction = {((np.abs(np.std(Z) ** 2 - np.std(X) ** 2)) / np.std(X) ** 2) * 100:.2f}%')

Integral estimate Control Variate: 1.71776, actual: 1.71828, error = 0.05%
Variance reduction = 17.18%


<h3>(4) Stratified sampling estimate </h3>

In [84]:
n_intervals = 10 # Number of intervals

W = np.zeros(n)

for i in range(n_intervals):
    temp = np.exp(i / n_intervals + U / n_intervals)
    W += temp

W = W / n_intervals

W_bar = np.mean(W)

error = np.abs(W_bar - actual_integral)

print(f'Integral estimate Control Variate: {W_bar:.5f}, actual: {actual_integral:.5f}, error = {error * 100:.2f}%')

Integral estimate Control Variate: 1.71823, actual: 1.71828, error = 0.01%


<h3>(5) Control variate for estimator in simulation </h3>

In [93]:
def simulate_blocking_system(arrival_intensity = 1, mean_service_time = 8, num_servers = 10, n = 10000, 
                             arrival_mode = 'poisson', service_mode = 'exponential', service_params = {}, variance_reduction = False):
    """
        Simulate simple blocking system with discrete events and no waiting room.
    """
    t_system = 0
    m = num_servers
    servers = np.zeros(m)
    blocked = 0 # Counter of number of blocked

    U_arrivals = np.random.uniform(0, 1, size = n)
    mu_u = 1/2
    X_arrivals = rvs_poisson(U, lambda_ = 1)

    c = np.cov(X_arrivals, U_arrivals)[0, 1] / np.var(U_arrivals)

    Z = X_arrivals + c * (U_arrivals - mu_u)

    for i in range(n):
        # Sample time from which this customer arrives
        if arrival_mode == 'poisson':
            # t_arrival = stats.expon.rvs(scale=arrival_intensity, size = 1)
            if variance_reduction:
                t_arrival = Z[i]
            else:
                t_arrival = stats.expon.rvs(scale=arrival_intensity, size = 1)
        elif arrival_mode == 'erlang':
            t_arrival = stats.erlang.rvs(a = 1, scale=arrival_intensity, size = 1)
        elif arrival_mode == 'hyper':
            t_arrival = rvs_hyperexponential(p = 0.8, lambda_1 = 0.8333, lambda_2 = 5.0)
        else:
            raise ValueError('Wrong arrival mode specified!')
        
        # Extend system time
        t_system += t_arrival

        # Find available server
        min_server_idx = np.argmin(servers)

        if t_system >= servers[min_server_idx]:
            if service_mode == 'exponential':
                t_service = stats.expon.rvs(scale=mean_service_time, size = 1)
            elif service_mode == 'constant':
                t_service = mean_service_time
            elif service_mode == 'pareto':
                k = service_params.get('k')
                t_service = rvs_pareto(mean_ = mean_service_time, k = k, size = 1)
            elif service_mode == 'normal':
                s = service_params.get('s')
                val = stats.norm.rvs(loc = mean_service_time, scale = s, size = 1)
                if val < 0:
                    val = 0
                t_service = val
            else:
                raise ValueError('Wrong service mode specified')
            servers[min_server_idx] = t_system + t_service
        else:
            blocked += 1

    # Compute blocked fraction
    blocked_fraction = blocked / n

    return blocked_fraction

def rvs_hyperexponential(p = 0.8, lambda_1 = 0.83, lambda_2 = 5.0, size = 1):    
    choices = stats.binom.rvs(p = 1 - p, n = 1, size = size)
    results = np.zeros(size)

    for i, choice in enumerate(choices):
        if choice == 0:
            results[i] =  stats.expon.rvs(scale = 1 / lambda_1, size = 1)    
        else:
            results[i] = stats.expon.rvs(scale = 1 / lambda_2, size = 1)

    return results

def rvs_poisson(U, lambda_ = 1):
    return -np.log(U) / lambda_


def rvs_pareto(mean_ = 8, k = 1.05, size = 1):
    # Find the value of Beta
    beta = mean_ * (k - 1) / k

    # Generate uniform numbers 
    U = np.random.uniform(0, 1, size = size)
    X = beta * (U ** (-1/k))

    return X

def confidence_interval(vals, alpha = 0.05):
    if type(vals) != np.ndarray:
        vals = np.array(vals)

    n = len(vals)

    mean_ = np.mean(vals)
    std_error = np.sqrt( 1 / (n - 1) * np.sum((vals - mean_) ** 2))

    t = stats.t.ppf(1 - (alpha / 2), df = n - 1 )

    conf = [mean_ - t * std_error / np.sqrt(n), mean_ + t * std_error / np.sqrt(n)]

    return np.array(conf)

def analytical_blocking_system(arrival_intensity = 1, mean_service_time = 8, num_servers = 10):
    lambda_ = arrival_intensity
    s = mean_service_time
    m = num_servers
    A = lambda_ * s

    temp = np.array([A ** i / np.math.factorial(i) for i in np.arange(0, m + 1, 1)])

    B = (A ** m / np.math.factorial(m)) / (temp.sum())

    return B


In [102]:


rounds = 10
fractions = np.zeros(rounds)
fractions_reduced = np.zeros(rounds)

print(f'Simulating program with and without variance reduction for {rounds} rounds')

for r in tqdm(range(rounds)):
    fractions[r] = simulate_blocking_system(variance_reduction=False)
    fractions_reduced[r] = simulate_blocking_system(variance_reduction=True)


conf_int1 = confidence_interval(fractions) * 100
conf_int2 = confidence_interval(fractions_reduced) * 100
print(f'Blocked fraction (No variance reduction) : [{conf_int1[0]:.3f}%, {conf_int1[1]:.3f}]%')
print(f'Blocked fraction (With variance reduction) : [{conf_int2[0]:.3f}%, {conf_int2[1]:.3f}]%')

Simulating program with and without variance reduction for 10 rounds


100%|██████████| 10/10 [00:09<00:00,  1.00it/s]

Blocked fraction (No variance reduction) : [11.637%, 12.641]%
Blocked fraction (With variance reduction) : [11.499%, 12.233]%


In [98]:
print(confidence_interval(fractions))
print(confidence_interval(fractions_reduced))

[0.11760616 0.12661384]
[0.11900719 0.12415281]
